In [4]:
with open('/home/wuct/ALICE/reps/hf-vn-working/correlations/PostProcessing/obj.txt', 'r') as f:
    Objs = [line.strip().replace('"', '') for line in f if line.strip()]

if "hCorrectedCorr_PtCand0to1_PtAssoc0to1_InvMassBin1" in Objs:
    print("Found the object in the list.")
else:
    print("Object not found in the list.")

Found the object in the list.


# construct mass vs v2 histogram

In [15]:
import ROOT
import json
import os
import numpy as np

file_mass_path = '/home/wuct/ALICE/reps/hf-vn-working/correlations/PostProcessing/InvMass/InvMassVsPt_010_.root'
file_v2Delta_path = '/home/wuct/ALICE/reps/hf-vn-working/correlations/PostProcessing/Output_CorrelationFitting_010_negDeta_Root/CorrPhiD0_FinalPlots.root'
json_path = '/home/wuct/ALICE/reps/hf-vn-working/correlations/PostProcessing/config_CorrAnalysis_v2_010_negDeta.json'
v2_delta_hh=0.7

# hMass.append(infile.Get(f'pt_{ptMin*10:.0f}_{ptMax*10:.0f}/hMassData'))
# hVn.append(infile.Get(f'pt_{ptMin*10:.0f}_{ptMax*10:.0f}/hVnVsMassData'))

def get_mass_distribution(MassVsPt, pt_min, pt_max):
    binMin = MassVsPt.GetYaxis().FindBin(pt_min*1.0001)
    binMax = MassVsPt.GetYaxis().FindBin(pt_max*0.9999)
    if pt_min != MassVsPt.GetYaxis().GetBinLowEdge(binMin):
        print(f"Warning: pt_min {pt_min} does not match bin edge {MassVsPt.GetYaxis().GetBinLowEdge(binMin)}")
    if pt_max != MassVsPt.GetYaxis().GetBinUpEdge(binMax):
        print(f"Warning: pt_max {pt_max} does not match bin edge {MassVsPt.GetYaxis().GetBinUpEdge(binMax)}")
    MassVsPt.GetYaxis().SetRange(binMin, binMax)
    proj = MassVsPt.ProjectionX(f"mass_proj_{pt_min}_{pt_max}")
    proj.SetDirectory(0)
    return proj
    

with open(json_path, 'r') as f:
    config = json.load(f)
ptBins = config['binsPtCandIntervals']
ptMins = ptBins[0:len(ptBins)-1]
ptMaxs = ptBins[1:len(ptBins)]
ptAssocBins = config['binsPtHadIntervals']
ptAssocMins = ptAssocBins[0:len(ptAssocBins)-1]
ptAssocMaxs = ptAssocBins[1:len(ptAssocBins)]
massBins = config['binsInvMassIntervals']
massMins = massBins[0:len(massBins)-1]
massMaxs = massBins[1:len(massBins)]

fileMass = ROOT.TFile.Open(file_mass_path)
hMassVsPt = fileMass.Get("hMassVsPt")

fileV2Delta = ROOT.TFile.Open(file_v2Delta_path)
hV2Deltas = {}
for obj in fileV2Delta.GetListOfKeys():
    hV2Deltas[obj.GetName()] = fileV2Delta.Get(obj.GetName())

# sort and slice histograms

for indexPtAssoc, (ptAssocMin, ptAssocMax) in enumerate(zip(ptAssocMins, ptAssocMaxs), start=1):
    mass_distributions = []
    v2_mass_distributions = []
    for indexPt, (ptMin, ptMax) in enumerate(zip(ptMins, ptMaxs), start=1):
        tmp_mass = hMassVsPt.Clone(f'tMass_PtCand{ptMin*10:02}to{ptMax*10:02}_PtAssoc{ptAssocMin*10:02}to{ptAssocMax*10:02}')
        mass_dist = get_mass_distribution(tmp_mass, ptMin, ptMax)
        mass_distributions.append(mass_dist)
        v2_mass_dist = ROOT.TH1F(f'v2Mass_PtCand{ptMin*10:02}to{ptMax*10:02}_PtAssoc{ptAssocMin*10:02}to{ptAssocMax*10:02}', f'v2Mass_PtCand{ptMin*10:02}to{ptMax*10:02}_PtAssoc{ptAssocMin*10:02}to{ptAssocMax*10:02}',len(massMins), np.array(massBins, dtype='d'))
        v2_mass_dist.SetTitle("Mass vs V2; Mass (GeV/c^2); V2")
        for indexMass, massMin in enumerate(massMins, start=1):
            hV2Delta_name = f"hv2Delta_PtBinAssoc{indexPtAssoc}_InvMassBin{indexMass}"
            hV2Delta = hV2Deltas.get(hV2Delta_name)
            valueV2Delta = hV2Delta.GetBinContent(indexPt)
            errorYV2Delta = hV2Delta.GetBinError(indexPt)
            v2_mass_dist.SetBinContent(indexMass, valueV2Delta)
            v2_mass_dist.SetBinError(indexMass, errorYV2Delta)
        v2_mass_dist.Scale(1/ROOT.TMath.Sqrt(v2_delta_hh))
        v2_mass_distributions.append(v2_mass_dist)
    # save histograms
    os.makedirs("/home/wuct/ALICE/reps/hf-vn-working/correlations/PostProcessing/Output_MassVsV2", exist_ok=True)
    outFileName = f"/home/wuct/ALICE/reps/hf-vn-working/correlations/PostProcessing/Output_MassVsV2/InvMassVsV2_PtAssoc{int(ptAssocMin*10):02}to{int(ptAssocMax*10):02}.root"
    outFile = ROOT.TFile.Open(outFileName, "RECREATE")
    for indexDist, (mass_dist, v2_mass_dist) in enumerate(zip(mass_distributions, v2_mass_distributions), start=1):
        outFile.mkdir(f'pt_{ptMins[indexDist-1]*10:.0f}_{ptMaxs[indexDist-1]*10:.0f}')
        outFile.cd(f'pt_{ptMins[indexDist-1]*10:.0f}_{ptMaxs[indexDist-1]*10:.0f}')
        mass_dist.Write('hMassData')
        v2_mass_dist.Write('hVnVsMassData')
    outFile.Close()
# hMass.append(infile.Get(f'pt_{ptMin*10:.0f}_{ptMax*10:.0f}/hMassData'))
# hVn.append(infile.Get(f'pt_{ptMin*10:.0f}_{ptMax*10:.0f}/hVnVsMassData'))



# perform the fits to extract v2 vs mass

In [ ]:
script = "/home/wuct/ALICE/reps/hf-vn-working/src/get_vn_vs_mass.py"
config_fit = "/home/wuct/ALICE/reps/hf-vn-working/configs/config_D0_correlation.yml"
input_mass_v2 = "/home/wuct/ALICE/reps/hf-vn-working/correlations/PostProcessing/Output_MassVsV2/InvMassVsV2_PtAssoc20to30.root"
batch = True
multitrial = False
os.system(f"python3 {script} {config_fit} {input_mass_v2} {'-b' if batch else ''} {'--multitrial' if multitrial else ''}")
# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description='Arguments')
#     parser.add_argument('fitConfigFileName', metavar='text', default='config_Ds_Fit.yml')
#     parser.add_argument('inFileName', metavar='text', default='')
#     parser.add_argument('--batch', '-b', help='suppress video output', action='store_true')
#     parser.add_argument('--multitrial', help='suppress reduntant prints', action='store_true')
#     args = parser.parse_args()

#     get_vn_vs_mass(
#         args.fitConfigFileName,
#         args.inFileName,
#         args.batch,
#         args.multitrial
#     )

0

Rebin from 500 bins to 250 bins -- Used bins = 500 in range 1-500

[INFO] Processing pt 0 - 1 GeV/c

--- First fit with only background on the side bands - Exclusion region = 4.00 sigma ---
 FCN=124.069 FROM MINOS     STATUS=SUCCESSFUL     12 CALLS          88 TOTAL
                     EDM=1.48955e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  BkgInt       3.04624e+03   3.61026e+00  -4.20486e-04  -4.08703e-08
   2  Slope       -6.03875e+00   1.17096e-02   1.17096e-02  -1.67645e-03
                               ERR DEF= 0.5

--- Estimate signal counts in the peak region ---
Compiled based function: funcbkg  based on a functor object.  Ndim = 1, Npar = 2
   ---> IntegralUnderHisto=764055.000000  IntegralUnderBkgFunc=771924.891436   EstimatedSignal=-7869.891436
   ---> (Tot-Bkg)/sqrt(Tot)=-9.003398 ---> Likely no signal
Abandon fit: no sig

In file included from input_line_56:1:
/home/wuct/ALICE/reps/hf-vn-working/src/../invmassfitter/VnVsMassFitter.cxx:640:19: warning: variable length arrays in C++ are a Clang extension [-Wvla-cxx-extension]
  Double_t SBbins[nMassBins];
                  ^~~~~~~~~
/home/wuct/ALICE/reps/hf-vn-working/src/../invmassfitter/VnVsMassFitter.cxx:640:19: note: initializer of 'nMassBins' is not a constant expression
/home/wuct/ALICE/reps/hf-vn-working/src/../invmassfitter/VnVsMassFitter.cxx:639:15: note: declared here
  const Int_t nMassBins = fVnVsMassHisto->GetNbinsX();
              ^
/home/wuct/ALICE/reps/hf-vn-working/src/../invmassfitter/VnVsMassFitter.cxx:1107:22: warning: variable length arrays in C++ are a Clang extension [-Wvla-cxx-extension]
    Double_t bkgpars[nBkgPars];
                     ^~~~~~~~
/home/wuct/ALICE/reps/hf-vn-working/src/../invmassfitter/VnVsMassFitter.cxx:1107:22: note: initializer of 'nBkgPars' is not a constant expression
/home/wuct/ALICE/reps/hf-vn-working/src